In [15]:
# Package that ensures a programatically interaction with operating system folder hierarchy.
from os import listdir

# Package used for clone a dictionary.
from copy import deepcopy

# Functions intended to extract some statistical parameters.
from numpy import max, std, average, sum, absolute

# With the following import we will be able to extract the linear regression parameters after 
# fitting experimental points to the model.
from scipy.stats import linregress

# biosignalsnotebooks own package that supports some functionalities used on the Jupyter Notebooks.
import biosignalsnotebooks as bsnb

# Python package that contains functions specialized on "Machine Learning" tasks.
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV, RFE
from sklearn.preprocessing import normalize

from json import loads, dump

# Package containing a diversified set of function for statistical processing and also provide support to array operations.
from numpy import max, array

# Python package that contains functions specialized on "Machine Learning" tasks.
from sklearn.neighbors import KNeighborsClassifier

# Python package that contains functions specialised on "Machine Learning" tasks.
from sklearn.model_selection import cross_val_score, LeaveOneOut

import numpy as np

from scipy.stats import linregress

import os

In [16]:
# Transposition of data from signal files to a Python dictionary.
relative_path = "/home/ferdinand/realworld_interact_systems"
data_folder = "/home/ferdinand/realworld_interact_systems/data"

# List of files (each file is a training example).
list_examples = listdir(data_folder)

In [17]:
# Initialization of dictionary.
signal_dict = {}

# Scrolling through each entry in the list.
for example in list_examples:
    if ".txt" in example: # Read only .txt files.
        # Get the class to which the training example under analysis belong.
        example_class = example.split("_")[0]

        # Get the trial number of the training example under analysis.
        example_trial = example.split("_")[1].split(".")[0]

        # Creation of a new "class" entry if it does not exist.
        if example_class not in signal_dict.keys():
            signal_dict[example_class] = {}

        # Load data.
        complete_data = bsnb.load(data_folder + "/" + example)

        # Store data in the dictionary.
        signal_dict[example_class][example_trial] = complete_data

In [18]:
# Channels (CH1 Flexor digitorum superficialis | CH2 Aductor policis | CH3 Accelerometer axis Z).
emg_flexor = "CH2"
emg_adductor = "CH4"
acc_z = "CH3"

In [19]:
# Clone "signal_dict".
features_dict = deepcopy(signal_dict)

# Navigate through "signal_dict" hierarchy.
list_classes = signal_dict.keys()
for class_i in list_classes:
    list_trials = signal_dict[class_i].keys()
    for trial in list_trials:
        # Initialise "features_dict" entry content.
        features_dict[class_i][trial] = []
        
        for chn in [emg_flexor, emg_adductor, acc_z]: ################################# order
            # Temporary storage of signal inside a reusable variable.
            signal = signal_dict[class_i][trial][chn]
            
            # Start the feature extraction procedure accordingly to the channel under analysis.
            if chn == emg_flexor or chn == emg_adductor: # EMG Features.
                # Converted signal (taking into consideration that our device is a "biosignalsplux", the resolution is
                # equal to 16 bits and the output unit should be in "mV").
                signal = bsnb.raw_to_phy("EMG", device="biosignalsplux", raw_signal=signal, resolution=16, option="mV")
                
                # Standard Deviation.
                features_dict[class_i][trial] += [std(signal)]
                # Maximum Value.
                features_dict[class_i][trial] += [max(signal)]
                # Zero-Crossing Rate.
                features_dict[class_i][trial] += [sum([1 for i in range(1, len(signal)) 
                                                       if signal[i]*signal[i-1] <= 0]) / (len(signal) - 1)]
                # Standard Deviation of the absolute signal.
                features_dict[class_i][trial] += [std(absolute(signal))]
            else: # ACC Features.
                # Converted signal (taking into consideration that our device is a "biosignalsplux", the resolution is
                # equal to 16 bits and the output unit should be in "g").
                signal = bsnb.raw_to_phy("ACC", device="biosignalsplux", raw_signal=signal, resolution=16, option="g")
                
                # Average value.
                features_dict[class_i][trial] += [average(signal)]
                # Standard Deviation.
                features_dict[class_i][trial] += [std(signal)]
                # Maximum Value.
                features_dict[class_i][trial] += [max(signal)]
                # Zero-Crossing Rate.
                features_dict[class_i][trial] += [sum([1 for i in range(1, len(signal)) 
                                                       if signal[i]*signal[i-1] <= 0]) / (len(signal) - 1)]
                # Slope of the regression curve.
                x_axis = range(0, len(signal))
                features_dict[class_i][trial] += [linregress(x_axis, signal)[0]]

In [20]:
# Package dedicated to the manipulation of json files.
from json import dump

filename = "classification_game_features.json"

# Generation of .json file in our previously mentioned "relative_path".
# [Generation of new file]
with open(relative_path + "/features/" + filename, 'w') as file:
    dump(features_dict, file)

# Load of data inside file storing it inside a Python dictionary.
with open(relative_path + "/features/" + filename) as file:
    features_dict = loads(file.read())

In [21]:
from sty import fg, rs
print(fg(98,195,238) + "\033[1mDict Keys\033[0m" + fg.rs + " define the class number")
print(fg(232,77,14) + "\033[1mDict Sub-Keys\033[0m" + fg.rs + " define the trial number\n")
print(features_dict)

Dict Keys define the class number
Dict Sub-Keys define the trial number

{'1': {'3': [0.0018116347807403438, -1.453216552734375, 0.0, 0.0018116347807403438, 1.9141502847591157e-05, -1.4767913818359375, 0.0, 1.9141502847591157e-05, -6.484591448648648, 0.0045966741355707415, -6.4518, 0.0, -4.923768513155715e-07], '4': [0.0012019167309204686, -1.4626922607421875, 0.0, 0.0012019167309204686, 2.1117077786380792e-05, -1.4767913818359375, 0.0, 2.1117077786380792e-05, -6.48106052962963, 0.0034059748614059244, -6.45239, 0.0, -1.1830831323264056e-06], '2': [0.0012319227810117907, -1.460540771484375, 0.0, 0.0012319227810117907, 2.040284509577354e-05, -1.4767913818359375, 0.0, 2.040284509577354e-05, -6.4832644684684695, 0.0039949995587272675, -6.42879, 0.0, -1.4844371327565837e-06], '1': [0.0013467510723971943, -1.4622802734375, 0.0, 0.0013467510723971943, 1.933381988871365e-05, -1.4767913818359375, 0.0, 1.933381988871365e-05, -6.483004716666666, 0.0051676954381953254, -6.44559, 0.0, -1.4928694723

In [22]:
# Initialisation of a list containing our training data and another list containing the labels of each training example.
features_list = []
class_training_examples = []

# Access each feature list inside dictionary.
list_classes = features_dict.keys()
for class_i in list_classes:
    list_trials = features_dict[class_i].keys()
    for trial in list_trials:
        # Storage of the class label.
        class_training_examples += [int(class_i)]
        features_list += [features_dict[class_i][trial]]

print(fg(232,77,14) + "\033[1m[Number of list entries;Number of sub-list entries]:\033[0m" + fg.rs + " [" + str(len(features_list)) + "; " + str(len(features_list[0])) + "]" + u'\u2713')
print(fg(253,196,0) + "\033[1mClass of each training example:\033[0m" + fg.rs)
print(class_training_examples)
print(fg(98,195,238) + "\033[1mFeatures List:\033[0m" + fg.rs)
print(features_list)


[Number of list entries;Number of sub-list entries]: [20; 13]✓
Class of each training example:
[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0]
Features List:
[[0.0018116347807403438, -1.453216552734375, 0.0, 0.0018116347807403438, 1.9141502847591157e-05, -1.4767913818359375, 0.0, 1.9141502847591157e-05, -6.484591448648648, 0.0045966741355707415, -6.4518, 0.0, -4.923768513155715e-07], [0.0012019167309204686, -1.4626922607421875, 0.0, 0.0012019167309204686, 2.1117077786380792e-05, -1.4767913818359375, 0.0, 2.1117077786380792e-05, -6.48106052962963, 0.0034059748614059244, -6.45239, 0.0, -1.1830831323264056e-06], [0.0012319227810117907, -1.460540771484375, 0.0, 0.0012319227810117907, 2.040284509577354e-05, -1.4767913818359375, 0.0, 2.040284509577354e-05, -6.4832644684684695, 0.0039949995587272675, -6.42879, 0.0, -1.4844371327565837e-06], [0.0013467510723971943, -1.4622802734375, 0.0, 0.0013467510723971943, 1.933381988871365e-05, -1.4767913818359375, 0.0, 1.933381988871365e-05,

In [23]:
feature_list_notnormalized = deepcopy(features_list)
max_per_feature = np.max(feature_list_notnormalized, axis=0)
np.save("max_per_feature.npy", max_per_feature)

features_list = normalize(features_list, axis=0, norm="max") # axis=0 specifies that each feature is normalised independently from the others 
                                                             # and norm="max" defines that the normalization reference value will be the feature maximum value.

In [24]:
# Creation of a "Support Vector Classifier" supposing that  our classes are linearly separable.
svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(5), scoring='accuracy')
# Fit data to the model.
selector = rfecv.fit(features_list, class_training_examples)
# Get list of average score of the virtual classifier
# avg_scores = rfecv.grid_scores_
avg_scores = rfecv.cv_results_['mean_test_score']
print(avg_scores)
max_score = max(avg_scores)
print(fg(98,195,238) + "\033[1mMaximum Average Score:\033[0m " + fg.rs + str(max_score))
for nbr_features in range(0, len(avg_scores)):
    if avg_scores[nbr_features] == max_score:
        optimal_nbr_features = nbr_features + 1
        break
print(fg(98,195,238) + "\033[1mOptimal Number of Features:\033[0m " + fg.rs + str(optimal_nbr_features))
bsnb.plot([range(1, len(rfecv.cv_results_['mean_test_score']) + 1)], [avg_scores], 
          y_axis_label="Cross validation score (nb of correct classifications)", x_axis_label="Number of features selected")

[0.7 0.8 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9]
Maximum Average Score: 0.9
Optimal Number of Features: 3


In [25]:
rfe = RFE(estimator=svc, step=1, n_features_to_select=optimal_nbr_features)

# Fit data to the model.
final_selector = rfe.fit(features_list, class_training_examples)

# Acception/Rejection Label attributed to each feature.
acception_labels = final_selector.support_

############### from webseite ###############
acception_labels_8 = [True, False, True, True, True, False, False, True, False, True, True, False, True]

print(fg(98,195,238) + "\033[1mRelevant Features (True):\033[0m " + fg.rs)
print(acception_labels)

Relevant Features (True): 
[False False False  True False False False False False  True False False
  True]


In [26]:
# Access each training example and exclude meaningless entries.
final_features_list = []
print(features_list.shape)
for example_nbr in range(0, len(features_list)):
    final_features_list += [list(array(features_list[example_nbr])[array(acception_labels_8)])]

(20, 13)


In [27]:
filename_final = "classification_game_features_final.json"

# Generation of .json file in our previously mentioned "relative_path".
# [Generation of new file]
with open(relative_path + "/features/" + filename_final, 'w') as file:
    dump({"features_list_final": final_features_list, "class_labels": class_training_examples}, file)

# Load of data inside file storing it inside a Python dictionary.
with open(relative_path + "/features/" + filename_final) as file:
    features_dict = loads(file.read())

from sty import fg, rs
print(fg(98,195,238) + "\033[1mDict Keys\033[0m" + fg.rs + " divides training example in its features and class label")
print(features_dict)

Dict Keys divides training example in its features and class label
{'features_list_final': [[0.5091278104239867, 0.0, 0.5091278104239867, 0.8404520279772982, 0.8404520279772982, 0.6302342538060254, -0.9960185809845946, -0.23819565741806295], [0.3377773710413215, 0.0, 0.3377773710413215, 0.9271942225138063, 0.9271942225138063, 0.46698155273817726, -0.9961096642424112, -0.5723365420851116], [0.34621003900777786, 0.0, 0.34621003900777786, 0.8958341815573521, 0.8958341815573521, 0.5477407124351794, -0.9924663339297486, -0.718121653745497], [0.3784804928483068, 0.0, 0.3784804928483068, 0.8488961532120278, 0.8488961532120278, 0.7085250296915919, -0.9950598911014744, -0.7222009411425864], [0.33048288386554187, 0.0, 0.33048288386554187, 0.869093208351423, 0.869093208351423, 0.44403578775340896, -0.9956465290331744, -0.44743716670387307], [0.718579393623026, 0.0, 0.718579393623026, 1.0, 1.0, 0.5483114597316746, -0.9873888282524828, 0.09322413040411497], [1.0, 0.0, 1.0, 0.8935410471535818, 0.893

KNN

In [28]:
training_examples = features_dict["features_list_final"]
class_training_examples = features_dict["class_labels"]

# k-Nearest Neighbour object initialisation.
knn_classifier = KNeighborsClassifier()

knn_classifier.fit(training_examples, class_training_examples)

print(training_examples)

print("Training data shape:", np.array(training_examples).shape)

[[0.5091278104239867, 0.0, 0.5091278104239867, 0.8404520279772982, 0.8404520279772982, 0.6302342538060254, -0.9960185809845946, -0.23819565741806295], [0.3377773710413215, 0.0, 0.3377773710413215, 0.9271942225138063, 0.9271942225138063, 0.46698155273817726, -0.9961096642424112, -0.5723365420851116], [0.34621003900777786, 0.0, 0.34621003900777786, 0.8958341815573521, 0.8958341815573521, 0.5477407124351794, -0.9924663339297486, -0.718121653745497], [0.3784804928483068, 0.0, 0.3784804928483068, 0.8488961532120278, 0.8488961532120278, 0.7085250296915919, -0.9950598911014744, -0.7222009411425864], [0.33048288386554187, 0.0, 0.33048288386554187, 0.869093208351423, 0.869093208351423, 0.44403578775340896, -0.9956465290331744, -0.44743716670387307], [0.718579393623026, 0.0, 0.718579393623026, 1.0, 1.0, 0.5483114597316746, -0.9873888282524828, 0.09322413040411497], [1.0, 0.0, 1.0, 0.8935410471535818, 0.8935410471535818, 0.5614618320654151, -0.9874351417734065, 0.26772981093175835], [0.8131023786

In [29]:
import numpy as np
from numpy import array

from bokeh.layouts import layout
from bokeh.models import CustomJS, Slider, Select, ColumnDataSource, WidgetBox
from bokeh.plotting import figure, show

tools = 'pan'
features_identifiers = ["std_emg_flexor", "zcr_emg_flexor", "std_abs_emg_flexor", "std_emg_adductor", "std_abs_emg_adductor", "std_acc_z", "max_acc_z", "m_acc_z"]

def slider():
    dict_features = {}
    for feature_nbr in range(0, len(training_examples[0])):
        values_feature = array(training_examples)[:, feature_nbr]
        
        # Fill of dict.
        for class_of_example in range(0, len(class_training_examples)):
            current_keys = list(dict_features.keys())
            if class_training_examples[class_of_example] not in current_keys:
                dict_features[class_training_examples[class_of_example]] = {}
            
            current_sub_keys = list(dict_features[class_training_examples[class_of_example]].keys())
            if features_identifiers[feature_nbr] not in current_sub_keys:
                dict_features[class_training_examples[class_of_example]][features_identifiers[feature_nbr]] = []
            
            dict_features[class_training_examples[class_of_example]][features_identifiers[feature_nbr]] += [values_feature[class_of_example]]
            
            # Add of two additional keys that will store the data currently being ploted.
            if feature_nbr == 0:
                if "x" not in current_sub_keys:
                    dict_features[class_training_examples[class_of_example]]["x"] = []
                dict_features[class_training_examples[class_of_example]]["x"] += [values_feature[class_of_example]]
            elif feature_nbr == 1:
                if "y" not in current_sub_keys:
                    dict_features[class_training_examples[class_of_example]]["y"] = []
                dict_features[class_training_examples[class_of_example]]["y"] += [values_feature[class_of_example]]
        
    source_class_0 = ColumnDataSource(data=dict_features[0])
    source_class_1 = ColumnDataSource(data=dict_features[1])
    source_class_2 = ColumnDataSource(data=dict_features[2])
    source_class_3 = ColumnDataSource(data=dict_features[3])

    plot = figure(x_range=(-1.5, 1.5), y_range=(-1.5, 1.5), tools='', toolbar_location=None, title="Pairing Classification Dimensions")
    bsnb.opensignals_style([plot])
    
    # Define different colours for points of each class.
    # [Class 0]
    plot.circle('x', 'y', source=source_class_0, line_width=3, line_alpha=0.6, color="red")
    # [Class 1]
    plot.circle('x', 'y', source=source_class_1, line_width=3, line_alpha=0.6, color="green")
    # [Class 2]
    plot.circle('x', 'y', source=source_class_2, line_width=3, line_alpha=0.6, color="orange")
    # [Class 3]
    plot.circle('x', 'y', source=source_class_3, line_width=3, line_alpha=0.6, color="blue")

    callback = CustomJS(args=dict(source=[source_class_0, source_class_1, source_class_2, source_class_3]), code="""
        // Each class has an independent data structure.
        var data_0 = source[0].data;
        var data_1 = source[1].data;
        var data_2 = source[2].data;
        var data_3 = source[3].data;
        
        // Selected values in the interface.
        var feature_identifier_x = x_feature.value;
        var feature_identifier_y = y_feature.value;
        console.log("x_feature: " + feature_identifier_x);
        console.log("y_feature: " + feature_identifier_y);
        
        // Update of values.
        var x_0 = data_0["x"];
        var y_0 = data_0["y"];
        for (var i = 0; i < x_0.length; i++) {
            x_0[i] = data_0[feature_identifier_x][i];
            y_0[i] = data_0[feature_identifier_y][i];
        }
        
        var x_1 = data_1["x"];
        var y_1 = data_1["y"];
        for (var i = 0; i < x_1.length; i++) {
            x_1[i] = data_1[feature_identifier_x][i];
            y_1[i] = data_1[feature_identifier_y][i];
        }
        
        var x_2 = data_2["x"];
        var y_2 = data_2["y"];
        for (var i = 0; i < x_2.length; i++) {
            x_2[i] = data_2[feature_identifier_x][i];
            y_2[i] = data_2[feature_identifier_y][i];
        }
        
        var x_3 = data_3["x"];
        var y_3 = data_3["y"];
        for (var i = 0; i < x_3.length; i++) {
            x_3[i] = data_3[feature_identifier_x][i];
            y_3[i] = data_3[feature_identifier_y][i];
        }
        
        // Communicate update.
        source[0].change.emit();
        source[1].change.emit();
        source[2].change.emit();
        source[3].change.emit();
    """)

    # x_feature_select = Select(title="Select the Feature of Axis x:", value="std_emg_flexor", options=features_identifiers, callback=callback)
    # callback.args["x_feature"] = x_feature_select
    
    # y_feature_select = Select(title="Select the Feature of Axis y:", value="zcr_emg_flexor", options=features_identifiers, callback=callback)
    # callback.args["y_feature"] = y_feature_select

    x_feature_select = Select(
        title="Select the Feature of Axis x:",
        value="std_emg_flexor",
        options=features_identifiers
    )
    y_feature_select = Select(
        title="Select the Feature of Axis y:",
        value="zcr_emg_flexor",
        options=features_identifiers
    )

    # Register your callback properly using js_on_change()
    callback.args["x_feature"] = x_feature_select
    callback.args["y_feature"] = y_feature_select

    x_feature_select.js_on_change("value", callback)
    y_feature_select.js_on_change("value", callback)

    widgets = WidgetBox(x_feature_select, y_feature_select)
    return [widgets, plot]

l = layout([slider(),], sizing_mode='scale_width')

show(l)

In [30]:
# A list with 8 arbitrary entries.
# test_examples_features = [0.65, 0.51]
test_examples_features = [0.65, 0.51, 0.70, 0.10, 0.20, 0.17, 0.23, 0.88]

# Classification.
print("Returned Class: ")
print(knn_classifier.predict([test_examples_features]))

# Probability of Accuracy.
print("Probability of each class:")
print(knn_classifier.predict_proba([test_examples_features]))

Returned Class: 
[2]
Probability of each class:
[[0.2 0.  0.8 0. ]]


In [31]:
# Load of data inside file, storing it inside a Python dictionary.
with open(relative_path + "/features/" + filename_final) as file:
    features_class_dict = loads(file.read())

features_class_dict.keys()

features_list = features_class_dict["features_list_final"]
class_training_examples = features_class_dict["class_labels"]
print(len(features_list[0]))

# Renaming original variable.
training_examples_a = features_list
print(len(features_list))

8
20


In [32]:
# k-Nearest Neighbour object initialisation.
knn_classifier_a = KNeighborsClassifier()

# Fit model to data.
knn_classifier_a.fit(training_examples_a, class_training_examples) 

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [33]:
leave_one_out_score_a = cross_val_score(knn_classifier_a, training_examples_a, class_training_examples, scoring="accuracy", cv=LeaveOneOut())

# Average accuracy of classifier.
mean_l1o_score_a = leave_one_out_score_a.mean()

# Standard Deviation of the previous estimate.
std_l1o_score_a = leave_one_out_score_a.std()

In [34]:
from sty import fg, rs
print(fg(232,77,14) + "\033[1mAverage Accuracy of Classifier:\033[0m" + fg.rs)
print(str(mean_l1o_score_a * 100) + " %")

print(fg(98,195,238) + "\033[1mStandard Deviation:\033[0m" + fg.rs)
print("+-" + str(round(std_l1o_score_a, 1) * 100) + " %")

Average Accuracy of Classifier:
95.0 %
Standard Deviation:
+-20.0 %


In [35]:
acception_labels_8 = np.array([True, False, True, True, True, False, False, True, False, True, True, False, True])


# --- Feature extraction for one recording ---
def extract_features(signal):
    """
    Extract the same 13 features as in training
    from one raw signal recording (.txt).
    """
    #####################
    emg_flexor_channel = 2 + 4
    emg_adductor_channel = 4 + 4
    acc_channel = 3 + 4

    signal = np.array(signal)

    # Ensure correct shape (n_samples, n_channels)
    if signal.ndim == 1:
        signal = signal.reshape(-1, 1)


    # Split channels
    emg_flexor = signal[:, emg_flexor_channel]  
    emg_adductor = signal[:, emg_adductor_channel] 
    acc_z = signal[:, acc_channel]

    # --- Convert raw to physical units
    emg_flexor_conv = bsnb.raw_to_phy("EMG", device="biosignalsplux", raw_signal=emg_flexor, resolution=16, option="mV")
    emg_adductor_conv = bsnb.raw_to_phy("EMG", device="biosignalsplux", raw_signal=emg_adductor, resolution=16, option="mV")
    acc_z_conv = bsnb.raw_to_phy("ACC", device="biosignalsplux", raw_signal=acc_z, resolution=16, option="g")

    features_emg_flexor = []
    features_emg_adductor = []
    for emg_signal in [emg_flexor_conv, emg_adductor_conv]:
        sigma_emg = np.std(emg_signal)
        max_emg = np.max(emg_signal)
        zcr_emg = np.sum(np.diff(np.sign(emg_signal)) != 0) / len(emg_signal)
        sigma_abs_emg = np.std(np.abs(emg_signal))
        if emg_signal is emg_flexor_conv:
            features_emg_flexor += [sigma_emg, max_emg, zcr_emg, sigma_abs_emg]
        if emg_signal is emg_adductor_conv:
            features_emg_adductor += [sigma_emg, max_emg, zcr_emg, sigma_abs_emg]


    # --- Accelerometer Z ---
    features_acc = []
    m_acc_z = np.mean(acc_z_conv)
    sigma_acc_z = np.std(acc_z_conv)
    max_acc_z = np.max(acc_z_conv)
    zcr_acc_z = np.sum(np.diff(np.sign(acc_z_conv)) != 0) / len(acc_z_conv)
    slope_acc_z = linregress(np.arange(len(acc_z_conv)), acc_z_conv)[0]
    features_acc += [m_acc_z, sigma_acc_z, max_acc_z, zcr_acc_z, slope_acc_z]

    # --- Combine all into one 13-feature vector ---
    features_13 = np.concatenate([features_emg_flexor, features_emg_adductor, features_acc])

    max_per_feature = np.load("max_per_feature.npy")
    # Compute the same scaling factors used in training
    normalized_features_13 = features_13 / (max_per_feature + 1e-12)

    return normalized_features_13


def classify_new_recording(filepath, classifier, feature_mask=acception_labels_8):
    """
    Reads one file, extracts features, selects the 8 important ones,
    and classifies with a trained model.
    """
    # Load raw signal
    signal = np.loadtxt(filepath)

    # Extract all 13 features
    full_features = extract_features(signal)

    # Select only the 8 relevant features
    reduced_features = np.array(full_features)[feature_mask]

    pred = classifier.predict([reduced_features])[0]

    return pred


def classify_all_in_folder(data_folder, classifier, feature_mask=acception_labels_8):
    """Classify all .txt files in the given folder."""
    results = []
    files = sorted([f for f in os.listdir(data_folder) if f.endswith(".txt")])

    for fname in files:
        fpath = os.path.join(data_folder, fname)
        try:
            pred = classify_new_recording(fpath, classifier, feature_mask)
            results.append((fname, pred))
            print(f"{fname:25s} → Predicted class: {pred}")
        except Exception as e:
            print(f"⚠️ Error processing {fname}: {e}")

    return results


data_folder = "/home/ferdinand/realworld_interact_systems/data"
results = classify_all_in_folder(data_folder, knn_classifier_a)

pred = classify_new_recording("/home/ferdinand/realworld_interact_systems/data/0_1.txt", knn_classifier_a)
print("Predicted class:", pred)


0_1.txt                   → Predicted class: 0
0_2.txt                   → Predicted class: 0
0_3.txt                   → Predicted class: 0
0_4.txt                   → Predicted class: 0
0_5.txt                   → Predicted class: 0
1_1.txt                   → Predicted class: 3
1_2.txt                   → Predicted class: 3
1_3.txt                   → Predicted class: 3
1_4.txt                   → Predicted class: 3
1_5.txt                   → Predicted class: 3
2_1.txt                   → Predicted class: 2
2_2.txt                   → Predicted class: 2
2_3.txt                   → Predicted class: 2
2_4.txt                   → Predicted class: 2
2_5.txt                   → Predicted class: 2
3_1.txt                   → Predicted class: 3
3_2.txt                   → Predicted class: 3
3_3.txt                   → Predicted class: 3
3_4.txt                   → Predicted class: 3
3_5.txt                   → Predicted class: 3
Predicted class: 0


In [36]:
print("Classifier expects:", knn_classifier_a.n_features_in_, "features")

Classifier expects: 8 features


In [37]:
import numpy as np
signal = np.loadtxt("/home/ferdinand/realworld_interact_systems/data/0_1.txt")
print(signal.shape)
print(signal[:3, :10])

(6150, 11)
[[  0.   0.   0.   0.   0.   0. 503. 609. 506.   0.]
 [  1.   0.   0.   0.   0.   0. 512. 610. 506.   0.]
 [  2.   0.   0.   0.   0.   0. 514. 610. 506.   0.]]


In [38]:
testdata = [0.35, 0.45, 0.60, 0.85, 0.25, 0.90, 0.30, 0.75]
print("Returned Class: ")
print(knn_classifier_a.predict([testdata]))

Returned Class: 
[2]


In [41]:
from pylsl import StreamInlet, resolve_byprop
import numpy as np
import time

# --- Load your trained model
import joblib
#knn_classifier = joblib.load("knn_classifier_a.pkl")

# --- Resolve LSL stream
print("# Looking for an OpenSignals stream...")
streams = resolve_byprop("name", "OpenSignals")
inlet = StreamInlet(streams[0])

# --- Real-time loop
window_size = 1000   # samples per classification window (~2 sec)
overlap = 500        # half overlap
buffer = []

while True:
    sample, timestamp = inlet.pull_sample()
    buffer.append(sample)

    if len(buffer) >= window_size:
        # Extract window
        signal = np.array(buffer[:window_size])
        buffer = buffer[overlap:]  # keep overlap for next window

        # Feature extraction + classification
        features = extract_features(signal)
        reduced_features = np.array(features)[acception_labels_8]
        pred = knn_classifier.predict([reduced_features])[0]

        print(f"Predicted class: {pred}")
        time.sleep(0.1)

# Looking for an OpenSignals stream...


2025-10-15 19:20:04.196 (  54.317s) [python          ]      netinterfaces.cpp:89    INFO| netif 'lo' (status: 1, multicast: 0, broadcast: 0)
2025-10-15 19:20:04.197 (  54.317s) [python          ]      netinterfaces.cpp:89    INFO| netif 'wlo1' (status: 1, multicast: 4096, broadcast: 2)
2025-10-15 19:20:04.197 (  54.317s) [python          ]      netinterfaces.cpp:89    INFO| netif 'lo' (status: 1, multicast: 0, broadcast: 0)
2025-10-15 19:20:04.197 (  54.317s) [python          ]      netinterfaces.cpp:89    INFO| netif 'wlo1' (status: 1, multicast: 4096, broadcast: 2)
2025-10-15 19:20:04.197 (  54.317s) [python          ]      netinterfaces.cpp:102   INFO| 	IPv4 addr: c0a80aaf
2025-10-15 19:20:04.197 (  54.317s) [python          ]      netinterfaces.cpp:89    INFO| netif 'lo' (status: 1, multicast: 0, broadcast: 0)
2025-10-15 19:20:04.197 (  54.317s) [python          ]      netinterfaces.cpp:89    INFO| netif 'wlo1' (status: 1, multicast: 4096, broadcast: 2)
2025-10-15 19:20:04.197 (  5

IndexError: index 8 is out of bounds for axis 1 with size 7

2025-10-15 19:22:50.012 ( 220.133s) [R_OpenSignals   ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...
2025-10-15 19:25:16.949 ( 367.070s) [R_OpenSignals   ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...
2025-10-15 19:26:16.604 ( 426.725s) [R_OpenSignals   ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...
2025-10-15 19:26:30.446 ( 440.567s) [R_OpenSignals   ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...
2025-10-15 19:30:55.853 ( 705.974s) [R_OpenSignals   ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...
2025-10-15 19:35:36.459 ( 986.580s) [R_OpenSignals   ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...
2025-10-15 19:38:14.289 (1144.410s) [R_O